# Variadic templates (Вариативные шаблоны)

Это шаблоны в C++, которые могут принимать переменное число аргументов.

Чтобы понять их смысл, попробуем реализовать функцию print из путона на C++.

Задача состоит в том, что необходимо создать такую функцию, в которую можно передать любое число параметров, поддерживающих вывод, и она выведет их все в консоль через " ", в конце стоит символ новой строки ('\n').

In [31]:
#include <iostream>

using namespace std;

Реализуем функцию, которая принимает на вход только 1 аргумент и выводит его:

In [39]:
template<typename T>
void print(T x) {
    cout << x << endl;
}

In [40]:
print(1); // работает

1


In [41]:
print(1, 2); // нельзя передать более однго аргумента

1
2


Теперь используем вариативный шаблон и реализуем функцию, 
которая принимает произвольное количество параметров.

In [42]:
template<typename T, typename... Args>
void print(T x, Args... args) {
    print(x);
    print(args...);
}

In [43]:
print(1);

1


In [44]:
print(1, 2, 3);

input_line_50:4:5: error: call to 'print' is ambiguous
    print(args...);
    ^~~~~
input_line_52:2:2: note: in instantiation of function template specialization '__cling_N540::print<int, int, int>' requested here
 print(1, 2, 3);
 ^
input_line_43:2:6: note: candidate function [with T = int, Args = <int>]
void print(T x, Args... args) {
     ^
input_line_50:2:6: note: candidate function [with T = int, Args = <int>]
void print(T x, Args... args) {
     ^


Interpreter Error: 

Всё работает, но как?!?!?

Образуется рекурсия, когда происходит вызов print(args...), и с каждым новым вызовом этой фунции аргументов в args становится всё меньше и меньше, т. к. постоянно первый элемент из args становится аргументом x.

Что здесь вообще происходит?

Например, просмотрим вызов функции print(1, 2, 3, 4, 5).
```cpp
print(1, 2, 3, 4, 5):
    // x = 1;
    // args = {2, 3, 4, 5};
    print(1); // вызов функции print с одним параметром, выводится 1
    print(args...); // == print(2, 3, 4, 5)

```
Теперь вызывается функция print(2, 3, 4, 5):
```cpp
print(2, 3, 4, 5):
    // x = 2;
    // args = {3, 4, 5};
    print(2); // выводится 2
    print(args...); // == print(3, 4, 5)
```
Далее происходит вызов функции print(3, 4, 5):
```cpp
print(3, 4, 5):
    // x = 3;
    // args = {4, 5};
    print(3); // вывод 3
    print(args...); // == print(4, 5)
```
Далее print(4, 5):
```cpp
print(4, 5):
    // x = 4;
    // args = {5}
    print(4); // вывод 4
    print(5); // здесь вызывается print с одним входным параметром, выводится 5
```
На этом моменте рекурсивные вызовы прекратились, 
возвращаемся по стеку вызовов к print(1, 2, 3, 4, 5) и выходим из функции.

Одно условие не выполнено, т. к. аргументы разделяются символом новой строки, 
а необходимо, чтобы разделителем был ' '.

Поэтому немного преобразуем нашу функцию.

Оставим print для одного аргумента без изменений.

In [8]:
template<typename T>
void print(T x) {
    cout << x << endl;
}

Теперь создадим _print для одного аргумента, который вместо символа новой строки будет выводить ' '.

In [9]:
template<typename T>
void _print(T x) {
    cout << x << ' ';
}

А сейчас перепишем print с вариативным шаблоном.

In [10]:
template<typename T, typename... Args>
void print(T x, Args... args) {
    _print(x);
    print(args...);
}

In [18]:
print(1, 2, 3, 4, 5); // работает при компилировании!!!

input_line_17:4:5: error: call to 'print' is ambiguous
    print(args...);
    ^~~~~
input_line_25:2:2: note: in instantiation of function template specialization '__cling_N510::print<int, int, int, int, int>' requested here
 print(1, 2, 3, 4, 5); // работает при компилировании!!!
 ^
input_line_12:2:6: note: candidate function [with T = int, Args = <int, int, int>]
void print(T x, Args... args) {
     ^
input_line_17:2:6: note: candidate function [with T = int, Args = <int, int, int>]
void print(T x, Args... args) {
     ^


Interpreter Error: 

Рассмотрим другой пример. Попробуем реализовать функцию подсчёта суммы переданных в функцию параметров.

Т. е. будет такая функция sum, при этом sum(1, 2, 3) == 6.

Сначала создадим функцию, которая будет возвращать сумму одного элемента.

In [19]:
template<typename T>
auto sum(T x) {
    return x;
}

Теперь сделаем функцию с вариативным шаблоном.

In [20]:
template<typename T, typename... Args>
auto sum(T x, Args... args) {
    return x + sum(args...);
}

In [21]:
sum(1, 2, 3); // работает при компилировании!!!

input_line_28:2:2: error: call to 'sum' is ambiguous
 sum(1, 2, 3); // работает при компилировании!!!
 ^~~
input_line_20:2:6: note: candidate function [with T = int, Args = <int, int>]
void sum(T x, Args... args) {
     ^
input_line_27:2:6: note: candidate function [with T = int, Args = <int, int>]
auto sum(T x, Args... args) {
     ^


Interpreter Error: 

Что происходит?

```cpp
sum(1, 2, 3, 4, 5):
    // x = 1;
    // args = {2, 3, 4, 5};
    return 1 + sum(2, 3, 4, 5):
        sum(2, 3, 4, 5):
            // x = 2;
            // args = {3, 4, 5};
            return 2 + sum(3, 4, 5):
                sum(3, 4, 5):
                    // x = 3;
                    // args = {4, 5};
                    return 3 + sum(4, 5):
                        sum(4, 5):
                            // x = 4;
                            // args = {5}
                            return 4 + sum(5):
                                sum(5):
                                    return 5;
```
Тогда вызов sum(1, 2, 3, 4, 5) превращается в return 1 + 2 + 3 + 4 + 5.

В C++17 эту функцию можно реализовать проще:

In [23]:
template<typename... T>
auto sum(T... args) {
    return (args + ...);
}

input_line_30:3:20: warning: pack fold expression is a C++17 extension [-Wc++17-extensions]
    return (args + ...);
                   ^


## Постановка эллипсиса (...)

Когда эллипсис ставился сразу после args, то он распаковывал args, 
т. е. передавал их не в виде контейнера, а как отдельные его части.

Но эллипсис также можно поставить не только после args. 
Например, если мы ходит применить к args какую-либо функцию.

In [24]:
template<typename T>
auto sqr(T x) {
    return x * x;
}

In [25]:
template<typename T>
auto p_sum(T x) {
    return x;
}

In [26]:
template<typename T, typename... Args>
auto p_sum(T x, Args... args) {
    return x + p_sum(sqr(args)...);
}

In [29]:
p_sum(1, 2, 3); // работает при компилировании!!!

input_line_33:3:16: error: call to 'sum' is ambiguous
    return x + sum(sqr(args)...);
               ^~~
input_line_36:2:2: note: in instantiation of function template specialization '__cling_N525::p_sum<int, int, int>' requested here
 p_sum(1, 2, 3); // работает при компилировании!!!
 ^
input_line_20:2:6: note: candidate function [with T = int, Args = <int>]
void sum(T x, Args... args) {
     ^
input_line_27:2:6: note: candidate function [with T = int, Args = <int>]
auto sum(T x, Args... args) {
     ^
input_line_30:2:6: note: candidate function [with T = <int, int>]
auto sum(T... args) {
     ^


Interpreter Error: 

Тогда получим следующие вызовы:
```cpp
p_sum(1, 2, 3):
    // x = 1;
    // args = {2, 3};
    return 1 + p_sum(sqr(args)...):
        p_sum(sqr(args) == p_sum(sqr(2), sqr(3)) == p_sum(4, 9):
            // x = 4;
            // args = {9};
            return 4 + p_sum(sqr(args)...):
                p_sum(sqr(args)...) == p_sum(sqr(9)) == p_sum(81):
                    // x = 5;
                    // args = {};
                    return 81;
```
В итоге получим: 1 + 4 + 81 или 1 + 2^2 + 3^4.